In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,mean,udf,sum,when
spark = SparkSession.builder \
    .appName("Typhoon Analyze") \
    .master("local[*]") \
    .getOrCreate()
spark.conf.set("spark.rapids.sql.enable","true")

data = spark.read.option("header", True).csv("../design/data/risk_assessment.csv")

24/12/24 06:56:49 WARN Utils: Your hostname, user-System-Product-Name resolves to a loopback address: 127.0.1.1; using 114.213.214.100 instead (on interface enp36s0f1)
24/12/24 06:56:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 06:56:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/24 06:56:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/24 06:56:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/12/24 06:56:51 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
data.show(5)

+--------+----+-----+---+----+--------+---------+--------------------+--------------------------------+
|storm_id|year|month|day|hour|latitude|longitude|               grade|Indicator of landfall or passage|
+--------+----+-----+---+----+--------+---------+--------------------+--------------------------------+
|    5101|1951|    2| 19|   6|    20.0|    138.5| Tropical Depression|                               0|
|    5101|1951|    2| 19|  12|    20.0|    138.5| Tropical Depression|                               0|
|    5101|1951|    2| 19|  18|    23.0|    142.1| Tropical Depression|                               0|
|    5101|1951|    2| 20|   0|    25.0|    146.0|Tropical Cyclone ...|                               0|
|    5101|1951|    2| 20|   6|    27.6|    150.6|Tropical Cyclone ...|                               0|
+--------+----+-----+---+----+--------+---------+--------------------+--------------------------------+
only showing top 5 rows



In [45]:

# 提取台风与时间的趋势
#columns year typhoon_count
typhoon_trend = data.groupBy("year").agg(count("storm_id").alias("typhoon_count")).orderBy("year")
typhoon_trend.show(5)

# 提取台风与季节的趋势
#columns season typhoon_count
season_trend = data.withColumn("season", when(col("month").isin(["12", "1", "2"]), "Winter")
                                      .when(col("month").isin(["3", "4", "5"]), "Spring")
                                      .when(col("month").isin(["6", "7", "8"]), "Summer")
                                      .when(col("month").isin(["9", "10", "11"]), "Fall")
                                      .otherwise("Unknown")) \
                   .groupBy("year", "season").agg(count("storm_id").alias("typhoon_count")).orderBy("year", "season")
season_trend.show(5)




+----+-------------+
|year|typhoon_count|
+----+-------------+
|1951|          792|
|1952|          836|
|1953|          864|
|1954|          727|
|1955|          858|
+----+-------------+
only showing top 5 rows

+----+------+-------------+
|year|season|typhoon_count|
+----+------+-------------+
|1951|  Fall|          273|
|1951|Spring|          184|
|1951|Summer|          239|
|1951|Winter|           96|
|1952|  Fall|          417|
+----+------+-------------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import first, lit, concat_ws,udf

# 提取台风与区域的趋势
# columns positon typoon_landed
region_trend = data.filter(col("Indicator of landfall or passage") == "1")

def to_point(la,lo):
        return [la,lo]

to_point_udf=udf(to_point)
region_trend = region_trend.withColumn('positon',to_point_udf(col('latitude'),col('longitude')))

region_trend = region_trend.groupBy('storm_id', 'year').agg(
    first("positon").alias("positon"),
).orderBy("year")

region_trend.show()



+--------+----+-------------+
|storm_id|year|      positon|
+--------+----+-------------+
|    9114|1991|[34.4, 137.7]|
|    9117|1991|[32.7, 129.7]|
|    9119|1991|[32.8, 129.7]|
|    9209|1992|[32.7, 133.1]|
|    9210|1992|[32.7, 130.5]|
|    9211|1992|[32.5, 131.9]|
|    9304|1993|[33.7, 134.7]|
|    9305|1993|[31.2, 131.0]|
|    9306|1993|[32.5, 129.7]|
|    9311|1993|[42.8, 144.2]|
|    9313|1993|[30.9, 130.3]|
|    9314|1993|[33.5, 135.3]|
|    9407|1994|[32.6, 132.8]|
|    9411|1994|[41.0, 140.2]|
|    9426|1994|[33.5, 135.3]|
|    9514|1995|[31.8, 130.2]|
|    9606|1996|[30.4, 130.5]|
|    9609|1996|[24.2, 123.7]|
|    9612|1996|[26.3, 127.9]|
|    9621|1996|[24.4, 125.4]|
+--------+----+-------------+
only showing top 20 rows



In [44]:
from pyspark.sql.functions import collect_list

# 将每年的 positon 合并成一个列表
region_trend_grouped = region_trend.groupBy("year") \
                                   .agg(collect_list("positon").alias("landings"))

region_trend_grouped.show(5)

+----+--------------------+
|year|            landings|
+----+--------------------+
|1991|[[34.4, 137.7], [...|
|1992|[[32.7, 133.1], [...|
|1993|[[33.7, 134.7], [...|
|1994|[[32.6, 132.8], [...|
|1995|     [[31.8, 130.2]]|
+----+--------------------+
only showing top 5 rows



In [51]:

from pyspark.sql.functions import concat_ws

# Convert the array column to a string
region_trend_grouped = region_trend_grouped.withColumn("landings", concat_ws(",", "landings"))
region_trend_grouped.show(5)

+----+--------------------+
|year|            landings|
+----+--------------------+
|1991|[34.4, 137.7],[32...|
|1992|[32.7, 133.1],[32...|
|1993|[33.7, 134.7],[31...|
|1994|[32.6, 132.8],[41...|
|1995|       [31.8, 130.2]|
+----+--------------------+
only showing top 5 rows



In [52]:

season_trend.coalesce(1).write.mode("overwrite").option("header",True).csv("result/llmdata/year_season")
region_trend_grouped.coalesce(1).write.mode("overwrite").option("header",True).csv("result/llmdata/landings")